All variables in input + MLP (which is my best model - but changing my best model parameters to Optuna 2 (without all variables in input without MLP) params 
                              since better than Optuna 1 and almost as good as all variables in input with MLP)

{'window': 7, 'n_layers': 4, 'lstm_units': 128, 'dropout_rate': 0.2, 'static_dense': 128, 'learning_rate': 0.0005728715834481826, 'batch_size': 64}

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 1. Loading libraries and datasets, and set up data
# ───────────────────────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import (Input, LSTM, Embedding, Flatten, Dense, Concatenate, SpatialDropout1D, BatchNormalization, Dropout, Add, RepeatVector)
from tensorflow.keras.models import Model
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

# Cluster, PH, Weather, Sales data
df = pd.read_csv(r" ... csv")

# Feature columns
time_varying_categorical_cols = ['Rain?','Name','Puasa','Public Holiday','Day','Month']  # varies with time
static_categorical_cols = ['Store_No','State','CODE (subcluster 1)','CODE FY26 1 (subcluster 2)','CODE FY26 2 (subcluster 3)']  # won't change depending on time
categorical_cols = time_varying_categorical_cols + static_categorical_cols
numeric_cols = ['Net_Amount','TC','Days_after_Opening','Average Daily Temperature (°C)']

# Filling in spaces to be encoded: 'CODE (subcluster 1)': blank, Name: no PH, Puasa: 0, Public Holiday: 0
# Filling empty spaces
df['CODE (subcluster 1)'] = df['CODE (subcluster 1)'].fillna('blank')
df['Name']               = df['Name'].fillna('no PH')
df['Puasa']              = df['Puasa'].fillna(0)
df['Public Holiday']     = df['Public Holiday'].fillna(0)

# Replace any empty‐string entries (''), if they exist
df['CODE (subcluster 1)'] = df['CODE (subcluster 1)'].replace('', 'blank')
df['Name']               = df['Name'].replace('', 'no PH')
df['Puasa']              = df['Puasa'].replace('', 0)
df['Public Holiday']     = df['Public Holiday'].replace('', 0)

# ───────────────────────────────────────────────────────────────────────────────
# 2. Encoding categorical columns
# ───────────────────────────────────────────────────────────────────────────────
# Splitting categorical columns into 2 types - to be embedded or be binary
embed_cols = []

for col in categorical_cols:
    n_uniques = df[col].nunique()
    if col in static_categorical_cols:
        embed_cols.append(col)
    elif n_uniques < 3:
        pass
    elif n_uniques >= 7:
        embed_cols.append(col)
    else:
        print(f"Column {col} has {n_uniques} categories: choose manually.")

# Encoding embed columns
encoders = {}
for col in embed_cols:
    le = LabelEncoder()
    df[col + '_enc'] = le.fit_transform(df[col])
    encoders[col] = le

# Binary columns - Puasa and Public Holiday are already in 0s and 1s
df['Rain?'] = df['Rain?'].map({'Yes': 1, ' No': 0})

# ───────────────────────────────────────────────────────────────────────────────
# 3. Scaling continuous features
# ───────────────────────────────────────────────────────────────────────────────
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# ───────────────────────────────────────────────────────────────────────────────
# 4. Building sequences for LSTM
# ───────────────────────────────────────────────────────────────────────────────
# purely numeric or binary (already scaled / mapped 0/1)
time_numeric_cols = [
    'Net_Amount',
    'TC',
    'Days_after_Opening',
    'Average Daily Temperature (°C)',
    'Rain?',          # make sure this is 0/1
    'Puasa',          # 0/1
    'Public Holiday'  # 0/1
]

# select window size 7-30 days
window = 7

X_num, X_name, X_day, X_month, X_stat, y = [], [], [], [], [], []

static_cols = [col + '_enc' for col in static_categorical_cols if col in embed_cols]

for store_id, grp in df.groupby('Store_No'):
    grp = grp.sort_values('Date')
    T   = len(grp)
    if T <= window: 
        continue

    # raw arrays
    arr_num    = grp[time_numeric_cols].values        # (T, n_num_feats)
    arr_name   = grp['Name_enc'].values               # (T,)
    arr_day    = grp['Day_enc'].values                # (T,)
    arr_month  = grp['Month_enc'].values              # (T,)
    arr_static = grp[static_cols].iloc[window:].values# (T-window, n_static_feats)
    arr_target = grp[['Net_Amount','TC']].values[window:]    # (T-window,)

    for i in range(T - window):
        X_num.append(   arr_num[i : i+window]     )  # → (window, n_num_feats)
        X_name.append(  arr_name[i : i+window]    )  # → (window,)
        X_day.append(   arr_day[i : i+window]     )
        X_month.append(arr_month[i : i+window]    )
        X_stat.append(  arr_static[i]             )  # → (n_static_feats,)
        y.append(       arr_target[i]             )  # → scalar

# stack into arrays
X_num    = np.stack(X_num)    # (N, window, n_num_feats)
X_name   = np.stack(X_name)   # (N, window)
X_day    = np.stack(X_day)    # (N, window)
X_month  = np.stack(X_month)  # (N, window)
X_stat   = np.stack(X_stat)   # (N, n_static_feats)
y        = np.array(y)        # (N,)

print(X_num.shape, X_name.shape, X_day.shape, X_month.shape, X_stat.shape, y.shape)

# ───────────────────────────────────────────────────────────────────────────────
# 5. Time-aware train-validation data split
# ───────────────────────────────────────────────────────────────────────────────
# Initialize empty Python lists to hold your training vs. validation sequences:
X_num_train, X_num_val, \
X_name_train, X_name_val, \
X_day_train,  X_day_val,  \
X_month_train,X_month_val,\
X_stat_train, X_stat_val, \
y_train,      y_val = ([] for _ in range(12))

for store_id, grp in df.groupby('Store_No'):
    grp = grp.sort_values('Date')
    T = len(grp)
    n_windows = T - window
    if n_windows <= 0: continue

    # how many windows to train on for this store
    split_store = int(0.8 * n_windows)

    arr_num    = grp[time_numeric_cols].values        # (T, n_num_feats)
    arr_name   = grp['Name_enc'].values               # (T,)
    arr_day    = grp['Day_enc'].values                # (T,)
    arr_month  = grp['Month_enc'].values              # (T,)
    arr_static = grp[static_cols].iloc[window:].values# (T-window, n_static_feats)
    arr_target = grp[['Net_Amount','TC']].values[window:]    # (T-window,)

    for i in range(n_windows):
        num_seq  = arr_num[i : i+window]
        name_seq = arr_name[i : i+window]
        day_seq  = arr_day[i : i+window]
        month_seq = arr_month[i : i+window]
        static = arr_static[i]
        target   = arr_target[i]
        if i < split_store:
            X_num_train.append(num_seq)
            X_name_train.append(name_seq)
            X_day_train.append(day_seq)
            X_month_train.append(month_seq)
            X_stat_train.append(static)
            y_train.append(target)
        else:
            X_num_val.append(num_seq)
            X_name_val.append(name_seq)
            X_day_val.append(day_seq)
            X_month_val.append(month_seq)
            X_stat_val.append(static)
            y_val.append(target)

# finally stack
X_num_train = np.stack(X_num_train)
X_name_train = np.stack(X_name_train)
X_day_train = np.stack(X_day_train)
X_month_train = np.stack(X_month_train)
X_stat_train = np.stack(X_stat_train)
y_train     = np.stack(y_train)

X_stat_train = np.array(X_stat_train)
X_stat_val   = np.array(X_stat_val)

# ------------------------------------------------------------
# 6.2  Same as (1) but with a 2-layer residual MLP on statics
# ------------------------------------------------------------
def build_sales_lstm(
        W, F, time_cardinalities, static_cardinalities,
        lstm_units   = 128,         # you can tune these
        dropout_rate = 0.2,
        static_dense = 128,
        learning_rate= 0.005728715834481826,
):
    
# --- time-series inputs ---------------------------------------------------
    num_in       = Input(shape=(W, F),           name='num_in')
    name_seq_in  = Input(shape=(W,), dtype='int32', name='name_seq_in')
    day_seq_in   = Input(shape=(W,), dtype='int32', name='day_seq_in')
    month_seq_in = Input(shape=(W,), dtype='int32', name='month_seq_in')

    dim = lambda n: min(100, n // 2 + 5)

    name_emb  = SpatialDropout1D(dropout_rate)(
                   Embedding(time_cardinalities['name'],  dim(time_cardinalities['name']))(name_seq_in))
    day_emb   = SpatialDropout1D(dropout_rate)(
                   Embedding(time_cardinalities['day'],   dim(time_cardinalities['day']))(day_seq_in))
    month_emb = SpatialDropout1D(dropout_rate)(
                   Embedding(time_cardinalities['month'], dim(time_cardinalities['month']))(month_seq_in))

    time_concat = Concatenate(axis=-1)([num_in, name_emb, day_emb, month_emb])

    # --- static branch: **deeper residual** ----------------------------------
    static_inputs, static_vecs = [], []
    for base, vocab in static_cardinalities:
        s_in = Input(shape=(1,), dtype='int32', name=f'{base}_in')  # ✅ correct
        s_emb = Embedding(vocab, dim(vocab))(s_in)
        s_emb = Flatten()(s_emb)
        static_inputs.append(s_in)
        static_vecs.append(s_emb)

    static_ctx = Concatenate()(static_vecs)             # concat raw embeddings

    # MLP block 1
    h = Dense(static_dense, 'relu')(static_ctx)
    h = BatchNormalization()(h)
    h = Dropout(dropout_rate)(h)
    # MLP block 2 with RESIDUAL skip
    h2 = Dense(static_dense, 'relu')(h)
    h2 = BatchNormalization()(h2)
    h2 = Dropout(dropout_rate)(h2)
    h  = Add()([h, h2])                                # residual connection

    # final compressed static context
    static_ctx = Dense(static_dense//2, 'relu')(h)
    static_ctx = BatchNormalization()(static_ctx)

    # ---- tile & state-init exactly like 6.1 ---------------------------------
    static_tile = RepeatVector(W)(static_ctx)
    time_concat = Concatenate(-1)([time_concat, static_tile])

    init_h = Dense(lstm_units, 'tanh')(static_ctx)
    init_c = Dense(lstm_units, 'tanh')(static_ctx)

    static_tile = RepeatVector(W)(static_ctx)
    time_concat = Concatenate(-1)([time_concat, static_tile])

    init_h = Dense(lstm_units, 'tanh')(static_ctx)
    init_c = Dense(lstm_units, 'tanh')(static_ctx)

    # —— now a 4-layer LSTM stack —— 
    # Layer 1, with initial state:
    x = LSTM(
        lstm_units,
        return_sequences=True,
        dropout=dropout_rate,
        )(time_concat, initial_state=[init_h, init_c])

    # Layer 2
    x = LSTM(
        lstm_units,
        return_sequences=True,
        dropout=dropout_rate,
        )(x)

    # Layer 3
    x = LSTM(
        lstm_units,
        return_sequences=True,
        dropout=dropout_rate,
        )(x)

    # Layer 4 (final), no return_sequences so it collapses to (batch, units)
    x = LSTM(
        lstm_units,
        return_sequences=False,
        dropout=dropout_rate,
        )(x)

    # now your normalization, dropout, dense, etc.
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(2, 'linear')(x) 

    model = Model(
        inputs=[num_in, name_seq_in, day_seq_in, month_seq_in] + static_inputs,
        outputs=out, name='Sales_LSTM_TiledResidual')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='mse', metrics=['mae'])
    return model


# Prepare cardinalities for embeddings
n_name  = df['Name_enc' ].nunique()
n_day   = df['Day_enc'  ].nunique()
n_month = df['Month_enc'].nunique()
time_cardinalities = {'name': n_name, 'day': n_day, 'month': n_month}

static_cardinalities = []
for enc_col in static_cols:                      # e.g. "Store_No_enc"
    base  = enc_col.replace('_enc','')           # "Store_No"
    vocab = df[enc_col].nunique()
    static_cardinalities.append((base, vocab))

# window & feature count
_, W, F = X_num.shape

# Cast *all* arrays to proper dtype and shape
X_num_train   = np.asarray(X_num_train,   dtype=np.float32)
X_num_val     = np.asarray(X_num_val,     dtype=np.float32)

def to_int2d(arr): return np.asarray(arr, dtype=np.int32)
X_name_train  = to_int2d(X_name_train)
X_day_train   = to_int2d(X_day_train)
X_month_train = to_int2d(X_month_train)
X_name_val    = to_int2d(X_name_val)
X_day_val     = to_int2d(X_day_val)
X_month_val   = to_int2d(X_month_val)

X_stat_train  = np.asarray(X_stat_train, dtype=np.int32)
X_stat_val    = np.asarray(X_stat_val,   dtype=np.int32)
y_train       = np.asarray(y_train,      dtype=np.float32)
y_val         = np.asarray(y_val,        dtype=np.float32)

# ───────────────────────────────────────────────────────────────────────────────
# 7. Fit Model
# ───────────────────────────────────────────────────────────────────────────────
# Build input dicts (by name ─ safest)
train_inputs = {
    'num_in':       X_num_train,
    'name_seq_in':  X_name_train,
    'day_seq_in':   X_day_train,
    'month_seq_in': X_month_train,
}
val_inputs   = {
    'num_in':       X_num_val,
    'name_seq_in':  X_name_val,
    'day_seq_in':   X_day_val,
    'month_seq_in': X_month_val,
}

# add static columns (need reshape (N,1))
for i, (base, _) in enumerate(static_cardinalities):
    train_inputs[f'{base}_in'] = X_stat_train[:, i].reshape(-1,1)
    val_inputs  [f'{base}_in'] = X_stat_val[:,   i].reshape(-1,1)

# Sanity check: print expected vs actual - make sure ValueError: Functional() don't pop up
print("\n── EXPECTED  vs  ACTUAL per‑sample shapes ──")
tmp_model = build_sales_lstm(W, F, time_cardinalities, static_cardinalities)
for inp in tmp_model.inputs:
    k   = inp.name.split(':')[0]
    exp = tuple(inp.shape[1:])
    act = tuple(train_inputs[k].shape[1:])
    ok  = "✅" if exp == act else "❌"
    print(f"{k:25s} expected={str(exp):12s} actual={str(act):12s} {ok}")
print("────────────────────────────────────────────\n")

# Instantiate final model & train
model = build_sales_lstm(W, F, time_cardinalities, static_cardinalities)
es   = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

history = model.fit(
    train_inputs, y_train,
    validation_data=(val_inputs, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[es, rlrp],
    verbose=2
)

# Plot training curves
plt.plot(history.history['loss'],     label='train')
plt.plot(history.history['val_loss'], label='val')
plt.xlabel('Epoch'); plt.ylabel('MSE Loss')
plt.legend(); plt.tight_layout(); plt.show()

# ───────────────────────────────────────────────────────────────────────────────
# 8.  Evaluate on a hold‑out test set and generate forecasts
# ───────────────────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Build a fresh 70/15/15 split Train/Val/Test (time‑aware per store) 
def time_aware_split_by_store(df, window, time_numeric_cols, static_cols):
    X_num_tr, X_num_va, X_num_te = [], [], []
    X_name_tr, X_name_va, X_name_te = [], [], []
    X_day_tr,  X_day_va,  X_day_te  = [], [], []
    X_month_tr,X_month_va,X_month_te= [], [], []
    X_stat_tr, X_stat_va, X_stat_te = [], [], []
    y_tr,      y_va,      y_te      = [], [], []

    for _, grp in df.groupby('Store_No'):
        grp = grp.sort_values('Date')
        T = len(grp)
        n_windows = T - window
        if n_windows <= 0: continue

        n_train = int(0.70 * n_windows)
        n_val   = int(0.15 * n_windows)
        # remainder → test
        n_test  = n_windows - n_train - n_val

        arr_num    = grp[time_numeric_cols].values
        arr_name   = grp['Name_enc' ].values
        arr_day    = grp['Day_enc'  ].values
        arr_month  = grp['Month_enc'].values
        arr_static = grp[static_cols].iloc[window:].values
        arr_target = grp[['Net_Amount','TC']].values[window:]

        for i in range(n_windows):
            seq_num   = arr_num[i:i+window]
            seq_name  = arr_name[i:i+window]
            seq_day   = arr_day[i:i+window]
            seq_month = arr_month[i:i+window]
            stat_vec  = arr_static[i]
            target    = arr_target[i]

            # buckets
            if i < n_train:
                bucket = (X_num_tr,  X_name_tr,  X_day_tr,  X_month_tr,  X_stat_tr,  y_tr)
            elif i < n_train + n_val:
                bucket = (X_num_va,  X_name_va,  X_day_va,  X_month_va,  X_stat_va,  y_va)
            else:
                bucket = (X_num_te,  X_name_te,  X_day_te,  X_month_te,  X_stat_te,  y_te)

            bucket[0].append(seq_num)
            bucket[1].append(seq_name)
            bucket[2].append(seq_day)
            bucket[3].append(seq_month)
            bucket[4].append(stat_vec)
            bucket[5].append(target)

    # stack to np arrays
    def _stack(lst): return np.stack(lst) if lst and isinstance(lst[0], np.ndarray) else np.array(lst)
    return tuple(map(_stack,
        [X_num_tr,X_num_va,X_num_te,
         X_name_tr,X_name_va,X_name_te,
         X_day_tr,X_day_va,X_day_te,
         X_month_tr,X_month_va,X_month_te,
         X_stat_tr,X_stat_va,X_stat_te,
         y_tr,y_va,y_te]))

# build split
(split_X_num_tr, split_X_num_va, split_X_num_te,
 split_X_name_tr, split_X_name_va, split_X_name_te,
 split_X_day_tr,  split_X_day_va,  split_X_day_te,
 split_X_month_tr,split_X_month_va,split_X_month_te,
 split_X_stat_tr, split_X_stat_va, split_X_stat_te,
 split_y_tr,      split_y_va,      split_y_te) = time_aware_split_by_store(
    df, window, time_numeric_cols, static_cols
)

# Assemble TEST input dict 
def build_input_dict(X_num,X_name,X_day,X_month,X_stat):
    d = {
        'num_in':       X_num.astype(np.float32),
        'name_seq_in':  X_name.astype(np.int32),
        'day_seq_in':   X_day.astype(np.int32),
        'month_seq_in': X_month.astype(np.int32)
    }
    for i,(base,_) in enumerate(static_cardinalities):
        d[f'{base}_in'] = X_stat[:,i].reshape(-1,1).astype(np.int32)
    return d

test_inputs = build_input_dict(
    split_X_num_te, split_X_name_te, split_X_day_te,
    split_X_month_te, split_X_stat_te)

# Predict on the test set
y_pred_scaled = model.predict(test_inputs, verbose=0)

# Inverse‑scale Net_Amount & TC
# remember scaler was fit on numeric_cols, where Net_Amount is col 0 and TC is col 1
# we’ll invert only those two columns
net_idx = numeric_cols.index('Net_Amount')
tc_idx  = numeric_cols.index('TC')

def inverse_scale(scaled_vec, col_idx):
    tmp      = np.zeros((len(scaled_vec), len(numeric_cols)))
    tmp[:,col_idx] = scaled_vec
    return scaler.inverse_transform(tmp)[:,col_idx]

net_pred = inverse_scale(y_pred_scaled[:,0], net_idx)
tc_pred  = inverse_scale(y_pred_scaled[:,1], tc_idx)

net_true = inverse_scale(split_y_te[:,0], net_idx)
tc_true  = inverse_scale(split_y_te[:,1], tc_idx)

# Metrics & Interpretation
def interpret(true, pred, name):
    # core metrics
    mae  = mean_absolute_error(true, pred)
    rmse = math.sqrt(mean_squared_error(true, pred))
    r2   = r2_score(true, pred)

    # reference stats
    mean_y = np.mean(true)
    std_y  = np.std(true)

    # relative ratios
    mae_ratio  = mae  / mean_y  if mean_y else float('nan')
    rmse_ratio = rmse / std_y   if std_y  else float('nan')

    # header
    print(f"\n=== {name} ===")
    print(f" MAE : {mae:8.2f}   (mean = {mean_y:8.2f}, MAE/mean = {mae_ratio:.2f})")
    print(f" RMSE: {rmse:8.2f}   (std  = {std_y:8.2f}, RMSE/std  = {rmse_ratio:.2f})")
    print(f" R²  : {r2:6.3f}")

    # interpret MAE‑to‑mean
    if mae_ratio < 0.10:
        print("   🔵 Excellent MAE (<10% of mean)")
    elif mae_ratio < 0.20:
        print("   🟢 Good MAE (<20% of mean)")
    elif mae_ratio < 0.30:
        print("   🟡 Acceptable MAE (<30% of mean)")
    else:
        print("   🔴 Poor MAE (>30% of mean)")

    # interpret RMSE‑to‑std
    if rmse_ratio < 0.50:
        print("   🔵 Excellent RMSE (<0.5 σ)")
    elif rmse_ratio < 0.75:
        print("   🟢 Good RMSE (<0.75 σ)")
    elif rmse_ratio < 1.00:
        print("   🟡 Acceptable RMSE (<1.0 σ)")
    else:
        print("   🔴 Poor RMSE (>1.0 σ)")

    # interpret R²
    if r2 >= 0.90:
        print("   🔵 Excellent R² (≥0.90)")
    elif r2 >= 0.75:
        print("   🟢 Good R² (0.75–0.90)")
    elif r2 >= 0.50:
        print("   🟡 Acceptable R² (0.50–0.75)")
    else:
        print("   🔴 Weak R² (<0.50)")

# run it for both targets
interpret(net_true, net_pred, "Net_Amount")
interpret(tc_true,  tc_pred,  "TC")

# Visual check for Net_Amount
fig, ax = plt.subplots(1,2, figsize=(12,4))

ax[0].scatter(net_true, net_pred, alpha=.4)
ax[0].plot([net_true.min(), net_true.max()],
           [net_true.min(), net_true.max()], 'k--')
ax[0].set_xlabel("True Net_Amount"); ax[0].set_ylabel("Predicted")
ax[0].set_title("Net_Amount – scatter")

ax[1].plot(net_true[:200], label='True')    # first 200 horizon windows
ax[1].plot(net_pred[:200], label='Pred', alpha=.7)
ax[1].set_title("Net_Amount – first 200 forecast windows")
ax[1].legend()

plt.tight_layout(); plt.show()

# Visual check for TC
fig, ax = plt.subplots(1,2, figsize=(12,4))

ax[0].scatter(tc_true, tc_pred, alpha=.4)
ax[0].plot([tc_true.min(), tc_true.max()],
           [tc_true.min(), tc_true.max()], 'k--')
ax[0].set_xlabel("True TC"); ax[0].set_ylabel("Predicted")
ax[0].set_title("TC – scatter")

ax[1].plot(tc_true[:200], label='True')    # first 200 horizon windows
ax[1].plot(tc_pred[:200], label='Pred', alpha=.7)
ax[1].set_title("TC – first 200 forecast windows")
ax[1].legend()

plt.tight_layout(); plt.show()